In [0]:
import pandas as pd
import csv
import re
import random
import numpy as np

from IPython import embed


In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1FNl79LtBEnhXx4NKoPubwThERuZC6WtP', dest_path='./Sentiment Analysis Dataset.zip', unzip=True)

Unzipping...Done.


In [0]:

gdd.download_file_from_google_drive(file_id='1ZXLowlLD161FlvLwcBjZ4TqAHxmiiket', dest_path='./processda1.zip', unzip=True)

Unzipping...Done.


In [0]:
def separate_dataset(filename):
    good_out = open("good_"+filename,"w+");
    bad_out  = open("bad_"+filename,"w+");


    with open(filename,'r') as f:
        reader = csv.reader(f)
        

        for line in reader:
           
            sentiment = line[1]
            sentence = line[3]

            if sentiment == "0":
                bad_out.write(sentence+"\n")
            elif sentiment == "1":
                good_out.write(sentence+"\n")


    good_out.close();
    bad_out.close();

separate_dataset("Sentiment Analysis Dataset.csv")

# Pre processing data 



In [0]:
!apt install -qq enchant
!pip install pyenchant
import re
import enchant
import itertools




dico = {}
dico1 = open('dicos/dico1.txt', 'rb')
for word in dico1:
    word = word.decode('utf8')
    word = word.split()
    dico[word[1]] = word[3]
dico1.close()
dico2 = open('dicos/dico2.txt', 'rb')
for word in dico2:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico2.close()
dico3 = open('dicos/dico3.txt', 'rb')
for word in dico3:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico3.close()

d = enchant.Dict('en_US')

def remove_repetitions(tweet):
    
    tweet=tweet.split()
    
    for i in range(len(tweet)):
        if tweet[i]=='<allcaps>': continue
        tweet[i]=''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet[i])).replace('#', '')
        if len(tweet[i])>0:
            if not d.check(tweet[i]):
                tweet[i] = ''.join(''.join(s)[:1] for _, s in itertools.groupby(tweet[i])).replace('#', '')
    tweet=' '.join(tweet)
    return tweet

def correct_spell(tweet):

    tweet = tweet.split()
    for i in range(len(tweet)):
        if tweet[i] in dico.keys():
            tweet[i] = dico[tweet[i]]
    tweet = ' '.join(tweet)
    return tweet


The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 8 not upgraded.
Need to get 1,300 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspel

In [0]:
!pip install regex

     |████████████████████████████████| 655kB 6.3MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609241 sha256=35bb4e6b38aa35be931d3179cf9739ec9149395adf068801329393f0e58e9293
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex


In [0]:
import sys
import regex as re

FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = " {} ".format(hashtag_body.lower())
    else:
        result = " <hashtag> " + " ".join([""] + [re.sub(r"([A-Z])",r" \1", hashtag_body, flags=FLAGS)])+" <allcaps> "
    return result




def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), " <smile> ")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), " <sadface> ")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), " <neutralface> ")
    text = re_sub(r"/"," / ")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat> ")
  #  text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")


    return text.lower()

In [0]:
def emoji_translation(text):
    loves = ["<3", "♥"]
    smilefaces = []
    sadfaces = []
    neutralfaces = []

    eyes = ["8",":","=",";"]
    nose = ["'","`","-",r"\\"]
    for e in eyes:
        for n in nose:
            for s in ["\)", "d", "]", "}","p"]:
                smilefaces.append(e+n+s)
                smilefaces.append(e+s)
            for s in ["\(", "\[", "{"]:
                sadfaces.append(e+n+s)
                sadfaces.append(e+s)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(e+n+s)
                neutralfaces.append(e+s)
            #reversed
            for s in ["\(", "\[", "{"]:
                smilefaces.append(s+n+e)
                smilefaces.append(s+e)
            for s in ["\)", "\]", "}"]:
                sadfaces.append(s+n+e)
                sadfaces.append(s+e)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(s+n+e)
                neutralfaces.append(s+e)

    smilefaces = list(set(smilefaces))
    sadfaces = list(set(sadfaces))
    neutralfaces = list(set(neutralfaces))

    t = []
    for w in text.split():
        if w in loves:
            t.append("<heart>")
        elif w in smilefaces:
            t.append("<smile>")
        elif w in neutralfaces:
            t.append("<neutralface>")
        elif w in sadfaces:
            t.append("<sadface>")
        else:
            t.append(w)
    newText = " ".join(t)
    return newText

In [0]:
#Clean Dataset
def clean_str(string):
  
    string= tokenize(string)
    string=emoji_translation(string)  

    #EMOJIS
    string = re.sub(r":\)"," <smile> ",string)
    string = re.sub(r":>"," <smile> ",string)
    string = re.sub(r" XD"," <smile> ",string)
    string = re.sub(r" <3"," <heart> ",string)
    string = re.sub(r" :p"," <lolface> ",string)
    string = re.sub(r" :P"," <lolface> ",string)
    string = re.sub(r":\("," <sadface> ",string)
    string = re.sub(r":<"," <sadface> ",string)
    string = re.sub(r":<"," <sadface> ",string)
    string = re.sub(r">:\("," <sadface> ",string)


    #STRANGE UNICODE \x...
    string = re.sub(r"\\x(\S)*","",string)


    string = re.sub(r"\s{2,}", " ", string)

    string = re.sub(r"i\'m", "i am", string)
    string = re.sub(r"\'re", "are", string)
    string = re.sub(r"he\'s", "he is", string)
    string = re.sub(r"it\'s", "it is", string)
    string = re.sub(r"that\'s", "that is", string)
    string = re.sub(r"who\'s", "who is", string)
    string = re.sub(r"what\'s", "what is", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = remove_repetitions(string)
    string = correct_spell(string)   
                  
    return string.strip().lower()

In [0]:
s=" #MachineLera lonnnn will :) looooooveeeee zooo #DIT #SickedLee I'm 2th    kkk"
print(clean_str(s))

<hashtag> machine lera <allcaps> lon will <smile> love zoo it <hashtag> sicked lee <allcaps> i am <number> to ok


In [0]:
def get_dataset(goodfile,badfile,limit,randomize=True):
    good_x = list(open(goodfile,"r").readlines())
    good_x = [s.strip() for s in good_x]
    
    bad_x  = list(open(badfile,"r").readlines())
    bad_x  = [s.strip() for s in bad_x]

    if (randomize):
        random.shuffle(bad_x)
        random.shuffle(good_x)

    good_x = good_x[:limit]
    bad_x = bad_x[:limit]

    x = good_x + bad_x
    x = [clean_str(s) for s in x]


    positive_labels = [[1] for _ in good_x]
    negative_labels = [[0] for _ in bad_x]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x,y]



In [0]:
def gen_batch(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

